# Keras Application 응용
- 4가지 모델의 결과를 종합하여 결과 도출

In [1]:
from glob import glob
glob('animals/*')

['animals\\bear.jpg',
 'animals\\cat.jpg',
 'animals\\crocodile.jpg',
 'animals\\dolphin.jpg',
 'animals\\eagle.jpg',
 'animals\\elephant.jpg',
 'animals\\lion.jpg',
 'animals\\penguin.jpg',
 'animals\\shark.jpg',
 'animals\\tree sparrow.jpg']

In [2]:
name_list = []
for file in glob('animals/*'):
    name = file.split('\\')[1]
    name = name.split('.')[0]
    name_list.append(name)
print(name_list)

['bear', 'cat', 'crocodile', 'dolphin', 'eagle', 'elephant', 'lion', 'penguin', 'shark', 'tree sparrow']


## All models and ensemble

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101

import numpy as np 
from PIL import Image

In [4]:
vgg16 = VGG16()
vgg19 = VGG19()
resnet50 = ResNet50()
resnet101 = ResNet101()

In [5]:
vgg16_list, vgg19_list, resnet50_list, resnet101_list = [],[],[],[]
ensemble = []
for file in glob('animals/*'):
    img = np.array(Image.open(file).resize((224,224)))
    yhat_v16 = vgg16.predict(img.reshape(-1, 224, 224, 3))
    yhat_v19 = vgg19.predict(img.reshape(-1, 224, 224, 3))
    yhat_r50 = resnet50.predict(img.reshape(-1, 224, 224, 3))
    yhat_v100 = resnet101.predict(img.reshape(-1, 224, 224, 3))
    yhat = (yhat_v16[0]+yhat_v19[0]+yhat_r50[0]+yhat_v100[0])/4.

    vgg16_list.append(decode_predictions(yhat_v16)[0][0][1])
    vgg19_list.append(decode_predictions(yhat_v19)[0][0][1])
    resnet50_list.append(decode_predictions(yhat_r50)[0][0][1])
    resnet101_list.append(decode_predictions(yhat_v100)[0][0][1])
    ensemble.append(decode_predictions(yhat.reshape(1,-1))[0][0][1])

## 데이터 프레임으로 합치기

In [6]:
import pandas as pd
df = pd.DataFrame({
    'Original':name_list, 'VGG16':vgg16_list, 'VGG19':vgg19_list,
    'ResNet50':resnet50_list, 'ResNet101':resnet101_list, 'Ensemble':ensemble
})
df

,Original,VGG16,VGG19,ResNet50,ResNet101,Ensemble
0,bear,brown_bear,ice_bear,brown_bear,brown_bear,brown_bear
1,cat,tabby,tabby,Egyptian_cat,Egyptian_cat,Egyptian_cat
2,crocodile,African_crocodile,African_elephant,cheetah,cheetah,cheetah
3,dolphin,dugong,hammerhead,sturgeon,Weimaraner,Weimaraner
4,eagle,bald_eagle,black_grouse,bald_eagle,vulture,bald_eagle
5,elephant,tusker,African_elephant,Weimaraner,African_elephant,African_elephant
6,lion,Saluki,lion,lion,lion,lion
7,penguin,king_penguin,king_penguin,king_penguin,king_penguin,king_penguin
8,shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark
9,tree sparrow,jay,jay,jay,jay,jay
